# Notebook for creating and saving Subject objects from datasets

In [14]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
abide_dir = os.path.dirname(os.path.dirname(cur_dir)) + '/abide/'
roi_200_dir = abide_dir + 'o_test_func_mean_ccs_nofilt_noglobal/Outputs/cpac/nofilt_noglobal/rois_cc200/'
os.path.exists(roi_200_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

## Subjects from ABIDE I preprocessed

In [15]:
phen_file = abide_dir + 'Phenotypic_V1_0b_preprocessed1.csv'
phen_df = pd.read_csv(phen_file)

In [16]:
subjects = list()
for index, row in phen_df.iterrows():
    data = list()
    file = row['FILE_ID'] + '_rois_cc200.1D'
    if(file in os.listdir(roi_200_dir)):
        with open(str(os.path.join(roi_200_dir, file)), newline='') as f:
            Lines = f.readlines()
            for l in Lines[1:]:
                ts = np.empty((200,))
                for n, i in enumerate(l.split("\t")):
                    ts[n] = i
                data.append(ts)       
        data_dict = {'roi_200_Cradd' : data}
        label_dict = {'dx_group': row['DX_GROUP']}
        s = Subject(row['SUB_ID'], row['SITE_ID'], row['SEX'], row['AGE_AT_SCAN'], data_dict, label_dict)
        subjects.append(s)
assert len(subjects) == len(os.listdir(roi_200_dir))

In [18]:
save_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
for subject in subjects:
    subject._save_subject(save_dir)

# Site TRs can be found on abide website or 
* https://www.researchgate.net/figure/SCANNING-PARAMETERS-OF-DIFFERENT-SITES-OF-ABIDE-1_tbl1_335722272

In [33]:
site_trs = {'CALTECH': 2, 'CMU': 2, 'KKI': '2.5', 'LEUVEN_1': 1.656, 'LEUVEN_2': 1.656, 'MAX_MUN': 3,
           'NYU': 2, 'OHSU': 2.5, 'OLIN': 2.5, 'PITT': 1.5, 'SBL': 2.2, 'SDSU': 2, 'STANFORD': 2, 
           'TRINITY': 2, 'UCLA_1': 3, 'UCLA_2': 3, 'UM_1': 2, 'UM_2': 2, 'USM': 2, 'YALE': 2}


In [36]:
import json

trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'
with open(trs_save_file, 'w') as fp:
    json.dump(site_trs, fp)